##### Import necessary libraries

In [ ]:
import numpy as np
from collections import defaultdict, Counter
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import math

##### Order Zero Markov Model - Main_0 function

In [4]:
def Main_0(positive_dataset, negative_dataset,k):

    # Step 1: Function to read fasta file
    def read_fasta(filename):
        sequences = []
        with open(filename, 'r') as file:
            for line in file:
                if not line.startswith('>'):
                    sequences.append(line.strip())
        return sequences

    # Step 2: Function to count nucleotide frequencies
    def count_nucleotide_frequencies(sequences):
        counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        total_bases = 0
        for seq in sequences:
            for base in seq:
                if base in counts:
                    counts[base] += 1
                    total_bases += 1
        return counts, total_bases

    # Step 3: Function to calculate nucleotide probabilities (MLE)
    def calculate_nucleotide_probabilities(counts, total_bases):
        return {base: count / total_bases for base, count in counts.items()}

    # Step 4: Function to calculate the probability of a sequence
    def calculate_sequence_probability(sequence, nucleotide_probs):
        prob = 1.0
        for base in sequence:
            prob *= nucleotide_probs.get(base, 1e-6)  # Use a small value for unknown nucleotides
        return prob

    # Step 5: Function to perform k-fold cross-validation
    def k_fold_cross_validation(positive_sequences, negative_sequences, k=5):
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        roc_data = {'train': [], 'test': []}
        pr_data = {'train': [], 'test': []}
        score_distributions = []

        for fold_num, (train_index, test_index) in enumerate(kf.split(positive_sequences)):
            train_positive = [positive_sequences[i] for i in train_index]
            test_positive = [positive_sequences[i] for i in test_index]
            train_negative = [negative_sequences[i] for i in train_index]
            test_negative = [negative_sequences[i] for i in test_index]

            # Train on k-1 folds
            pos_counts, pos_total_bases = count_nucleotide_frequencies(train_positive)
            neg_counts, neg_total_bases = count_nucleotide_frequencies(train_negative)

            pos_probs = calculate_nucleotide_probabilities(pos_counts, pos_total_bases)
            neg_probs = calculate_nucleotide_probabilities(neg_counts, neg_total_bases)

            # Collect scores for training and testing datasets
            train_scores = {'positive': [], 'negative': []}
            test_scores = {'positive': [], 'negative': []}

            # Training set scoring
            for seq in train_positive:
                pos_prob = calculate_sequence_probability(seq, pos_probs)
                neg_prob = calculate_sequence_probability(seq, neg_probs)
                score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('inf')
                train_scores['positive'].append(score)

            for seq in train_negative:
                pos_prob = calculate_sequence_probability(seq, pos_probs)
                neg_prob = calculate_sequence_probability(seq, neg_probs)
                score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('-inf')
                train_scores['negative'].append(score)

            # Test set scoring
            for seq in test_positive:
                pos_prob = calculate_sequence_probability(seq, pos_probs)
                neg_prob = calculate_sequence_probability(seq, neg_probs)
                score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('inf')
                test_scores['positive'].append(score)

            for seq in test_negative:
                pos_prob = calculate_sequence_probability(seq, pos_probs)
                neg_prob = calculate_sequence_probability(seq, neg_probs)
                score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('-inf')
                test_scores['negative'].append(score)

            # Store score distributions
            score_distributions.append({'fold': fold_num + 1, 'train': train_scores, 'test': test_scores})

            # Calculate ROC and PR curves for training set
            y_train_true = np.array([1] * len(train_positive) + [0] * len(train_negative))
            y_train_scores = np.array(train_scores['positive'] + train_scores['negative'])
            fpr_train, tpr_train, _ = roc_curve(y_train_true, y_train_scores)
            roc_auc_train = auc(fpr_train, tpr_train)
            roc_data['train'].append((fpr_train, tpr_train, roc_auc_train))

            precision_train, recall_train, _ = precision_recall_curve(y_train_true, y_train_scores)
            pr_auc_train = auc(recall_train, precision_train)
            pr_data['train'].append((precision_train, recall_train, pr_auc_train))

            # Calculate ROC and PR curves for test set
            y_test_true = np.array([1] * len(test_positive) + [0] * len(test_negative))
            y_test_scores = np.array(test_scores['positive'] + test_scores['negative'])
            fpr_test, tpr_test, _ = roc_curve(y_test_true, y_test_scores)
            roc_auc_test = auc(fpr_test, tpr_test)
            roc_data['test'].append((fpr_test, tpr_test, roc_auc_test))

            precision_test, recall_test, _ = precision_recall_curve(y_test_true, y_test_scores)
            pr_auc_test = auc(recall_test, precision_test)
            pr_data['test'].append((precision_test, recall_test, pr_auc_test))

        return roc_data, pr_data, score_distributions

    # Step 6: Plot ROC Curves
    def plot_roc_curves(roc_data, dataset_type='test'):
        plt.figure()
        for fold_num, (fpr, tpr, roc_auc) in enumerate(roc_data[dataset_type]):
            plt.plot(fpr, tpr, label=f'Fold {fold_num + 1} (AUC = {roc_auc:.2f})')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curves for Each Fold - {dataset_type.capitalize()} Set')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        # Save the ROC curves plot
        plt.savefig(f'roc_curves_{dataset_type.lower()}.png')
        plt.show()

    # Step 7: Plot Precision-Recall Curves
    def plot_precision_recall_curves(pr_data, dataset_type='test'):
        plt.figure()
        for fold_num, (precision, recall, pr_auc) in enumerate(pr_data[dataset_type]):
            plt.plot(recall, precision, label=f'Fold {fold_num + 1} (AUC = {pr_auc:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curves for Each Fold - {dataset_type.capitalize()} Set')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        # Save the Precision-Recall curves plot
        plt.savefig(f'precision_recall_curves_{dataset_type.lower()}.png')
        plt.show()

    # Step 8: Plot Score Distributions
    def plot_score_distributions(score_distributions, dataset_type='test'):
        for fold_data in score_distributions:
            fold_num = fold_data['fold']
            plt.figure()
            plt.hist(fold_data[dataset_type]['positive'], bins=30, alpha=0.5, label='Positive', color='blue')
            plt.hist(fold_data[dataset_type]['negative'], bins=30, alpha=0.5, label='Negative', color='red')
            plt.xlabel('Score')
            plt.ylabel('Frequency')
            plt.title(f'Score Distributions for Fold {fold_num} - {dataset_type.capitalize()} Set')
            plt.legend(loc='upper right')
            plt.grid(True)
            plt.show()

            # Save the plot for each fold
            plt.savefig(f'fold_{fold_num}_score_distribution.png')
            plt.show()

    # Main function to run the pipeline
    def main(positive_fasta, negative_fasta, k=5):
        positive_sequences = read_fasta(positive_fasta)
        negative_sequences = read_fasta(negative_fasta)
        
        roc_data, pr_data, score_distributions = k_fold_cross_validation(
            positive_sequences, negative_sequences, k
        )
        
        # Plot ROC and Precision-Recall curves for both test and train datasets
        plot_roc_curves(roc_data, 'test')
        plot_roc_curves(roc_data, 'train')
        
        plot_precision_recall_curves(pr_data, 'test')
        plot_precision_recall_curves(pr_data, 'train')
        
        # Plot Score distributions for each fold separately
        plot_score_distributions(score_distributions, 'test')
    
    # Force a delay to allow memory changes to reflect
    time.sleep(0.1)

    final_memory = get_memory_usage()
    print(f"Final memory usage (order 0): {final_memory:.2f} MB")
    print(f"Memory used for order 0: {final_memory - initial_memory:.2f} MB")

    return main(positive_dataset, negative_dataset, k)


##### Order M Markov Model - For any order other than 0

In [5]:
def Main_M(positive_dataset,negative_dataset,k,M):

    # Reading FASTA sequences
    def read_fasta(fasta_file):
        sequences = []
        with open(fasta_file, 'r') as f:
            sequence = ''
            for line in f:
                if line.startswith('>'):
                    if sequence:
                        sequences.append(sequence)
                        sequence = ''
                else:
                    sequence += line.strip()
            if sequence:
                sequences.append(sequence)
        return sequences

    # Transition counting for Markov model of order M
    def count_transitions(sequences, M):
        transition_counts = defaultdict(Counter)
        pair_counts = Counter()

        for sequence in sequences:
            for i in range(len(sequence) - M):
                pair = sequence[i:i+M]
                next_nuc = sequence[i+M]
                transition_counts[pair][next_nuc] += 1
                pair_counts[pair] += 1

        return transition_counts, pair_counts

    # Calculating transition probabilities for Markov model of order M
    def calculate_transition_probabilities(transition_counts, pair_counts):
        transition_probabilities = defaultdict(dict)
        for pair, next_nucs in transition_counts.items():
            for nuc, count in next_nucs.items():
                transition_probabilities[pair][nuc] = count / pair_counts[pair]
        return transition_probabilities

    # Calculating sequence probability based on Markov model of order M
    def calculate_sequence_probability(sequence, transition_probabilities,M):
        probability = 1.0
        for i in range(len(sequence) - M):
            pair = sequence[i:i+M]
            next_nuc = sequence[i+M]
            if pair in transition_probabilities and next_nuc in transition_probabilities[pair]:
                probability *= transition_probabilities[pair][next_nuc]
            else:
                probability *= 1e-4  # Smoothing for unseen pairs
        return probability

    # Updated function to perform k-fold cross validation and save counts to a text file
    def k_fold_cross_validation(positive_sequences, negative_sequences, k, M):
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        results = []
        roc_data = []
        pr_data = []
        train_roc_data = []
        train_pr_data = []
        
        with open('fold_counts.txt', 'w') as f_out:
            f_out.write('Fold\tTP\tFN\tFP\tTN\n')  # Header line

            for fold_num, (train_index, test_index) in enumerate(kf.split(positive_sequences)):
                train_positive = [positive_sequences[i] for i in train_index]
                test_positive = [positive_sequences[i] for i in test_index]
                train_negative = [negative_sequences[i] for i in train_index]
                test_negative = [negative_sequences[i] for i in test_index]

                # Count transitions and calculate probabilities for positive and negative training data
                pos_transition_counts, pos_pair_counts = count_transitions(train_positive, M)
                neg_transition_counts, neg_pair_counts = count_transitions(train_negative, M)

                pos_transition_probs = calculate_transition_probabilities(pos_transition_counts, pos_pair_counts)
                neg_transition_probs = calculate_transition_probabilities(neg_transition_counts, neg_pair_counts)

                # Get scores for test data
                test_scores = {
                    'TP': [],
                    'TN': [],
                    'FP': [],
                    'FN': []
                }

                # Positive test sequences
                for seq in test_positive:
                    pos_prob = calculate_sequence_probability(seq, pos_transition_probs,M)
                    neg_prob = calculate_sequence_probability(seq, neg_transition_probs,M)
                    score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('inf')
                    if score > 0:
                        test_scores['TP'].append(score)
                    else:
                        test_scores['FN'].append(score)

                # Negative test sequences
                for seq in test_negative:
                    pos_prob = calculate_sequence_probability(seq, pos_transition_probs,M)
                    neg_prob = calculate_sequence_probability(seq, neg_transition_probs,M)
                    score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('-inf')
                    if score < 0:
                        test_scores['TN'].append(score)
                    else:
                        test_scores['FP'].append(score)

                results.append(test_scores)

                # Write counts to the text file
                f_out.write(f'{fold_num + 1}\t{len(test_scores["TP"])}\t{len(test_scores["FN"])}\t{len(test_scores["FP"])}\t{len(test_scores["TN"])}\n')

                # Calculate ROC and PRC for test data
                y_true_test = np.array([1] * len(test_positive) + [0] * len(test_negative))
                y_scores_test = np.array(test_scores['TP'] + test_scores['FN'] + test_scores['FP'] + test_scores['TN'])
                fpr, tpr, _ = roc_curve(y_true_test, y_scores_test)
                roc_auc = auc(fpr, tpr)
                roc_data.append((fpr, tpr, roc_auc))

                precision, recall, _ = precision_recall_curve(y_true_test, y_scores_test)
                pr_auc = auc(recall, precision)
                pr_data.append((precision, recall, pr_auc))

                # Calculate ROC and PRC for training data (training on training)
                train_scores = {
                    'TP': [],
                    'TN': [],
                    'FP': [],
                    'FN': []
                }

                # Positive training sequences
                for seq in train_positive:
                    pos_prob = calculate_sequence_probability(seq, pos_transition_probs,M)
                    neg_prob = calculate_sequence_probability(seq, neg_transition_probs,M)
                    score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('inf')
                    if score > 0:
                        train_scores['TP'].append(score)
                    else:
                        train_scores['FN'].append(score)

                # Negative training sequences
                for seq in train_negative:
                    pos_prob = calculate_sequence_probability(seq, pos_transition_probs,M)
                    neg_prob = calculate_sequence_probability(seq, neg_transition_probs,M)
                    score = math.log(pos_prob / neg_prob) if neg_prob > 0 else float('-inf')
                    if score < 0:
                        train_scores['TN'].append(score)
                    else:
                        train_scores['FP'].append(score)

                # Calculate ROC and PRC for training data
                y_true_train = np.array([1] * len(train_positive) + [0] * len(train_negative))
                y_scores_train = np.array(train_scores['TP'] + train_scores['FN'] + train_scores['FP'] + train_scores['TN'])
                fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
                roc_auc_train = auc(fpr_train, tpr_train)
                train_roc_data.append((fpr_train, tpr_train, roc_auc_train))

                precision_train, recall_train, _ = precision_recall_curve(y_true_train, y_scores_train)
                pr_auc_train = auc(recall_train, precision_train)
                train_pr_data.append((precision_train, recall_train, pr_auc_train))

        return results, roc_data, pr_data, train_roc_data, train_pr_data

    def plot_score_distributions(results, M):
        for fold_num, fold_results in enumerate(results):
            plt.figure()
            # Combine scores from positive and negative test datasets
            positive_scores = fold_results['TP'] + fold_results['FN']  # Positive dataset scores
            negative_scores = fold_results['FP'] + fold_results['TN']  # Negative dataset scores

            plt.hist(positive_scores, bins=50, alpha=0.5, label='Positive', color='blue')
            plt.hist(negative_scores, bins=50, alpha=0.5, label='Negative', color='red')
            
            plt.title(f'Score Distribution for Fold {fold_num + 1} (MM Order {M})')
            plt.xlabel('Score')
            plt.ylabel('Frequency')
            plt.legend(loc='best')
            plt.grid(True)
            plt.savefig(f'score_distribution_fold_{fold_num + 1}_MM_Order_{M}.png')
            plt.show()

    # Plotting ROC curves
    def plot_roc_curves(roc_data, title_suffix, M):
        plt.figure()
        for i, (fpr, tpr, roc_auc) in enumerate(roc_data):
            plt.plot(fpr, tpr, label=f'Fold {i + 1} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title(f'ROC Curves - {title_suffix} Data (MM Order {M})')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.savefig(f'roc_curves_{title_suffix.lower()}_MM_order_{M}.png')
        plt.show()

    # Plotting Precision-Recall curves
    def plot_precision_recall_curves(pr_data, title_suffix, M):
        plt.figure()
        for i, (precision, recall, pr_auc) in enumerate(pr_data):
            plt.plot(recall, precision, label=f'Fold {i + 1} (AUC = {pr_auc:.2f})')
        plt.title(f'Precision-Recall Curves - {title_suffix} Data (MM Order {M})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.savefig(f'precision_recall_curves_{title_suffix.lower()}_MM_order_{M}.png')
        plt.show()

    # Main function to run the entire pipeline
    def main(positive_fasta, negative_fasta, k, M):
        positive_sequences = read_fasta(positive_fasta)
        negative_sequences = read_fasta(negative_fasta)
        
        results, roc_data, pr_data, train_roc_data, train_pr_data = k_fold_cross_validation(
            positive_sequences, negative_sequences, k, M
        )
        
        # Output score distributions
        plot_score_distributions(results, M)
        
        # Plot ROC and Precision-Recall curves for test data
        plot_roc_curves(roc_data, "Testing", M)
        plot_precision_recall_curves(pr_data, "Testing", M)
        
        # Plot ROC and Precision-Recall curves for training data
        plot_roc_curves(train_roc_data, "Training", M)
        plot_precision_recall_curves(train_pr_data, "Training", M)
    
    return main(positive_dataset, negative_dataset, k, M)

##### Execution of Main Functions

In [ ]:
positive_fasta = r'path\to\your\file' # Promoter sequence(s)
negative_fasta = r'path\to\your\file' # Non-promoter sequence(s)

# Function to safely get an integer input
def get_int_input(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Please enter a valid integer.")

# Get inputs
M = get_int_input("Markov model order: ")
k = get_int_input("Fold number 'k' for cross validation: ")

# Use the correct variable 'M' and 'k'
if M == 0:
    Main_0(positive_fasta, negative_fasta, k) 
else:
    Main_M(positive_fasta, negative_fasta, k, M)
